# GraphSAGE
GraphSAGE 包含采样和聚合（Sample and aggregate），首先使用节点之间连接信息，对邻居进行采样，然后通过多层聚合函数不断地将相邻节点的信息融合在一起。用融合后的信息预测节点标签。下图展示了 GraphSAGE 的聚合过程，采用了两层聚合层

<img src="../img/graphsage.png" width="800" >

## 算法流程

1. 在整体的图数据上，从某一个中心节点开始采样，得到一个 k 阶的子图，示意图中给出的示例是一个二阶子图
2. 有了这个二阶子图，我们可以先利用 GNN 把二阶的邻接点聚合成一阶的邻接点（图 2 中绿色的部分），再把一阶的邻接点聚合成这个中心节点（图 2 中蓝色的部分）
3. 有了聚合好的这个中心节点的 Embedding，我们就可以去完成一个预测任务，比如这个中心节点的标签是被点击的电影，那我们就可以让这个 GNN 完成一个点击率预估任务

<img src="../img/graphsage3.png" width="700" >

## 聚合函数

**Mean aggregator**

$$\pmb{h}_v^k \leftarrow \mathrm{MEAN}(\{\pmb{h}_u^{k-1},\forall u \in \mathcal{N}(v)\})$$

**GCN aggregator**

$$\pmb{h}_v^k \leftarrow \sigma (\pmb{W} \cdot \mathrm{MEAN}(\{\pmb{h}_v^{k-1}\} \cup \{\pmb{h}_u^{k-1},\forall u \in \mathcal{N}(v)\}))$$

**LSTM aggregator**

LSTM 有比较好的抽取特征能力，因此可以使用了 LSTM 进行聚合，但是因为节点之间没有明显的顺序关系，因此需要打乱之后放入 LSTM

**Pooling aggregator**

$$\pmb{h}_v^k \leftarrow \max (\{\sigma(\pmb{W} \pmb{h}_u^{k-1} + \pmb{b}),\forall u \in \mathcal{N}(v)\})$$

## 目标函数
GraphSAGE 可以采用无监督训练或者有监督训练

无监督训练采用负采样算法，公式如下

$$J_G(\pmb{z}_u) = - \log (\sigma(\pmb{z}_u^\top \pmb{z}_v)) - Q \cdot \mathbb{E}_{v_n \ \sim P_n \ (v)} \log (\sigma(-\pmb{z}_u^\top \pmb{z}_{v_n}))$$

其中 $P_n$ 是负采样的概率分布，$Q$ 是负样本的数量，$\pmb{z}_u$ 通过聚合顶点的邻接点特征产生

监督学习形式根据任务的不同直接设置目标函数即可，如最常用的节点分类任务使用交叉熵损失函数